In [2]:
pip install pandas-profiling

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 34.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 34.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27082 sha256=133d96c0b20755b72c089ef3e8c6a3c3aa1c6b21d02c542d9b821ddf34116dd2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ea/1c/a8/5cec3479cd45136a7111e2d96aac299b219b199c411665250b
Successfully built htmlmin
Note: you may need to restart the kernel to use updated packages.


In [32]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
#import pandas_profiling as pp

from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.metrics import roc_curve, classification_report, confusion_matrix

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'MarketingData.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the training data file
marketing_data = pd.read_csv(file_content_stream)
marketing_data.head()

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen,Channel
0,6623.613537,5513.093240,6019.057354,5669.568008,5898.660607,5179.234947,2
1,5642.542497,5829.866565,3960.339943,4270.020548,3498.818262,4327.423268,2
2,5292.078175,6634.370556,4444.335138,4888.286021,3265.391352,4887.560190,2
3,5595.227928,4754.860698,2977.856511,3462.490957,3609.264559,4268.641413,0
4,5126.693267,6009.649079,3811.569943,4744.115976,3829.516831,5097.491872,2


In [17]:
## reporting the number of observations in each marketing channel
marketing_data.shape[0]

20000

### Model Building: Random Forest
***

In [6]:
## defining inputs and targets
x = marketing_data.drop(columns = 'Channel')
y = marketing_data['Channel']

## splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [25]:
### One vs One Classifier, One vs Rest Classifier
one_vs_one_rf = OneVsOneClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(x_train, y_train)
one_vs_all_rf = OneVsRestClassifier(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3)).fit(x_train, y_train)

## predicting on the testing dataset
one_vs_one_rf_preds = one_vs_one_rf.predict(x_test)
one_vs_all_rf_preds = one_vs_all_rf.predict_proba(x_test)

## changing likelihoods to labels
## one_vs_one_rf_preds = np.argmax(one_vs_one_rf_preds, axis = 1) # don't need this statement with .predict statements
one_vs_all_rf_preds = np.argmax(one_vs_all_rf_preds, axis = 1) # got rid of the + 1 statement

## printing classification report ONE V ONE
print('One vs One:')
print(classification_report(y_test, one_vs_one_rf_preds))

## printing classification report ONE V REST
print('One vs All:')
print(classification_report(y_test, one_vs_all_rf_preds))

One vs One:
              precision    recall  f1-score   support

           0       0.77      0.72      0.74      1001
           1       0.65      0.76      0.70      1000
           2       0.85      0.63      0.72       998
           3       0.78      0.90      0.83      1001

    accuracy                           0.75      4000
   macro avg       0.76      0.75      0.75      4000
weighted avg       0.76      0.75      0.75      4000

One vs All:
              precision    recall  f1-score   support

           0       0.74      0.72      0.73      1001
           1       0.77      0.63      0.69      1000
           2       0.72      0.81      0.76       998
           3       0.80      0.87      0.83      1001

    accuracy                           0.76      4000
   macro avg       0.76      0.76      0.75      4000
weighted avg       0.76      0.76      0.75      4000



In [ ]:
### why does one vs one not require predict_proba?

### why does one vs one not require the np.argmax function?

### Model Building: CatBoost
***

In [36]:
### One vs One Classifier, One vs Rest Classifier
one_vs_one_rf = OneVsOneClassifier(estimator = CatBoostClassifier(n_estimators = 500, max_depth = 3, verbose = False, learning_rate = 0.1)).fit(x_train, y_train)
one_vs_all_rf = OneVsRestClassifier(estimator = CatBoostClassifier(n_estimators = 500, max_depth = 3, verbose = False, learning_rate = 0.1)).fit(x_train, y_train)

## predicting on the testing dataset
one_vs_one_rf_preds = one_vs_one_rf.predict(x_test)
one_vs_all_rf_preds = one_vs_all_rf.predict_proba(x_test)

## changing likelihoods to labels
## one_vs_one_rf_preds = np.argmax(one_vs_one_rf_preds, axis = 1) # don't need this statement with .predict statements
one_vs_all_rf_preds = np.argmax(one_vs_all_rf_preds, axis = 1) # got rid of the + 1 statement

## printing classification report ONE V ONE
print('One vs One:')
print(classification_report(y_test, one_vs_one_rf_preds))

## printing classification report ONE V REST
print('One vs All:')
print(classification_report(y_test, one_vs_all_rf_preds))

One vs One:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1001
           1       0.86      0.84      0.85      1000
           2       0.91      0.92      0.92       998
           3       0.89      0.88      0.89      1001

    accuracy                           0.89      4000
   macro avg       0.89      0.89      0.89      4000
weighted avg       0.89      0.89      0.89      4000

One vs All:
              precision    recall  f1-score   support

           0       0.88      0.89      0.89      1001
           1       0.85      0.83      0.84      1000
           2       0.91      0.91      0.91       998
           3       0.88      0.89      0.89      1001

    accuracy                           0.88      4000
   macro avg       0.88      0.88      0.88      4000
weighted avg       0.88      0.88      0.88      4000



Based on the above models, I would chose to do **One vs One with CatBoostClassifier** (supplemental information found here: https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier) because it received an F1 score of 0.9, compared to the One vs All CatBoostClassifier (0.89 F1 score), One vs One Random Forest (0.74 F1 score), and One vs All Random Forest (0.73 F1 score). 